# Buscando os Dados do CNES #

## O codigo abaixo pega os dados direto to FTP do CNES e tranforma em um DF do Pandas.

A função **read_dbc_geopandas** é baseada na função read_dbc, porém ela usa o geopandas para fazer a leitura do DBF, pois o dbfread não estava conseguindo abrir os DBFs do CNES.

In [1]:
!pip install pysus
!pip install geopandas

from datetime import datetime
from tempfile import NamedTemporaryFile
import geopandas as gpd
import os
from ftplib import FTP
from pysus.utilities.readdbc import read_dbc, dbc2dbf
from dbfread import DBF
import pandas as pd
from pysus.online_data import CACHEPATH

In [2]:
group_dict = {
        "LT" :  ["Leitos - A partir de Dez/2019", 12, 2019],
}

def download(group: str, state: str, year: int, month: int, cache: bool=True) -> object:
    """
    Download CNES records for group, state, year and month and returns dataframe
    :param group: 
        LT – Leitos - A partir de Dez/2019
    :param month: 1 to 12
    :param state: 2 letter state code
    :param year: 4 digit integer
    """
    #state = state.upper()
    year2 = str(year)[-2:]
    month = str(month).zfill(2)
    input_date = datetime(int(year), int(month), 1)
    avaiable_date = datetime(group_dict[group][2], group_dict[group][1], 1)
    if input_date < avaiable_date:
        raise ValueError(f"CNES does not contain data for {group_dict[group][1]}")
    ftp = FTP('ftp.datasus.gov.br')
    ftp.login()
    if input_date >= avaiable_date:
        ftype = 'DBC'
        ftp.cwd('dissemin/publicos/CNES/200508_/Dados/{}/'.format(group))
        fname = '{}{}{}{}.dbc'.format(group, state, str(year2).zfill(2), month)
    cachefile = os.path.join(CACHEPATH, 'CNES_' + fname.split('.')[0] + '_.parquet')
    if os.path.exists(cachefile):
        df = pd.read_parquet(cachefile)
        return df
    df = _fetch_file(fname, ftp, ftype)
    if cache:
        df.to_parquet(cachefile)
    return df

def _fetch_file(fname, ftp, ftype):
    try:
        ftp.retrbinary('RETR {}'.format(fname), open(fname, 'wb').write)
    except:
        raise Exception("File {} not available".format(fname))
    if ftype == 'DBC':
        df = read_dbc_geopandas(fname, encoding='iso-8859-1')
    elif ftype == 'DBF':
        dbf = DBF(fname, encoding='iso-8859-1')
        df = pd.DataFrame(list(dbf))
    os.unlink(fname)
    return df

def read_dbc_geopandas(filename,encoding='utf-8'):
    """
    Opens a DATASUS .dbc file and return its contents as a pandas
    Dataframe, using geopandas
    :param filename: .dbc filename
    :param encoding: encoding of the data
    :return: Pandas Dataframe.
    """
    if isinstance(filename, str):
        filename = filename
    with NamedTemporaryFile(delete=False) as tf:
        out = tf.name + '.dbf'
        dbc2dbf(filename, out)
        dbf = gpd.read_file(out, encoding=encoding).drop("geometry",axis=1)
        df = pd.DataFrame(dbf)
    os.unlink(tf.name)

    return df 

In [3]:
LT_GO_DEZ_2019 = download('LT',"GO", 2019, 12)

In [4]:
LT_GO_DEZ_2019.head()

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,NIV_HIER,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR
0,9331603,520010,None,None,None,None,M,3,00000000000000,3,...,None,None,2,33,9,0,9,0,201912,1244
1,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,2,41,2,0,2,0,201912,2062
2,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,1,03,2,0,1,1,201912,2062
3,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,4,10,1,0,1,0,201912,2062
4,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,6,34,4,0,3,1,201912,2062


In [8]:
GO_01_2020 = download('LT',"GO", 2020, 1)
GO_02_2020 = download('LT',"GO", 2020, 2)
GO_03_2020 = download('LT',"GO", 2020, 3)
GO_04_2020 = download('LT',"GO", 2020, 4)
GO_05_2020 = download('LT',"GO", 2020, 5)
GO_06_2020 = download('LT',"GO", 2020, 6)
GO_07_2020 = download('LT',"GO", 2020, 7)
GO_08_2020 = download('LT',"GO", 2020, 8)
GO_09_2020 = download('LT',"GO", 2020, 9)
GO_10_2020 = download('LT',"GO", 2020, 10)
GO_11_2020 = download('LT',"GO", 2020, 11)
GO_12_2020 = download('LT',"GO", 2020, 12)

In [9]:
LT_GO_2020 = pd.concat([GO_01_2020,GO_01_2020,GO_02_2020,GO_03_2020,GO_04_2020,
                        GO_05_2020,GO_06_2020,GO_07_2020,GO_08_2020,GO_09_2020,
                        GO_10_2020,GO_11_2020, GO_12_2020], axis=0)

In [10]:
LT_GO_2020.head()

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,NIV_HIER,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR
0,9331603,520010,None,None,None,None,M,3,00000000000000,3,...,None,None,2,33,9,0,9,0,202001,1244
1,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,4,43,3,0,3,0,202001,2062
2,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,1,03,2,0,1,1,202001,2062
3,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,2,41,2,0,2,0,202001,2062
4,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,6,34,4,0,3,1,202001,2062


In [11]:
GO_01_2021 = download('LT',"GO", 2021, 1)
GO_02_2021 = download('LT',"GO", 2021, 2)
GO_03_2021 = download('LT',"GO", 2021, 3)
GO_04_2021 = download('LT',"GO", 2021, 4)
GO_05_2021 = download('LT',"GO", 2021, 5)
GO_06_2021 = download('LT',"GO", 2021, 6)
GO_07_2021 = download('LT',"GO", 2021, 7)
GO_08_2021 = download('LT',"GO", 2021, 8)
GO_09_2021 = download('LT',"GO", 2021, 9)
GO_10_2021 = download('LT',"GO", 2021, 10)
GO_11_2021 = download('LT',"GO", 2021, 11)
GO_12_2021 = download('LT',"GO", 2021, 12)

In [12]:
LT_GO_2021 = pd.concat([GO_01_2021,GO_01_2021,GO_02_2021,GO_03_2021,GO_04_2021,
                        GO_05_2021,GO_06_2021,GO_07_2021,GO_08_2021,GO_09_2021,
                        GO_10_2021,GO_11_2021, GO_12_2021], axis=0)

In [13]:
LT_GO_2021.head()

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,NIV_HIER,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR
0,9331603,520010,None,None,None,None,M,3,00000000000000,3,...,None,None,2,33,9,0,9,0,202101,1244
1,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,6,34,4,0,3,1,202101,2062
2,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,1,03,2,0,1,1,202101,2062
3,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,5,45,3,0,3,0,202101,2062
4,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,4,43,3,0,3,0,202101,2062


In [14]:
GO_01_2022 = download('LT',"GO", 2022, 1)
GO_02_2022 = download('LT',"GO", 2022, 2)
GO_03_2022 = download('LT',"GO", 2022, 3)
GO_04_2022 = download('LT',"GO", 2022, 4)
GO_05_2022 = download('LT',"GO", 2022, 5)
GO_06_2022 = download('LT',"GO", 2022, 6)
GO_07_2022 = download('LT',"GO", 2022, 7)
GO_08_2022 = download('LT',"GO", 2022, 8)
GO_09_2022 = download('LT',"GO", 2022, 9)
GO_10_2022 = download('LT',"GO", 2022, 10)
GO_11_2022 = download('LT',"GO", 2022, 11)

In [15]:
LT_GO_2022 = pd.concat([GO_01_2022,GO_01_2022,GO_02_2022,GO_03_2022,GO_04_2022,
                        GO_05_2022,GO_06_2022,GO_07_2022,GO_08_2022,GO_09_2022,
                        GO_10_2022,GO_11_2022], axis=0)

In [16]:
LT_GO_2022.head()

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,NIV_HIER,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR
0,9331603,520010,None,None,None,None,M,3,00000000000000,3,...,None,None,2,33,9,0,9,0,202201,1244
1,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,2,41,2,0,2,0,202201,2062
2,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,1,03,2,0,1,1,202201,2062
3,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,6,34,4,0,3,1,202201,2062
4,2335506,520013,15,None,None,None,M,3,00269860000125,1,...,None,None,4,43,3,0,3,0,202201,2062


In [21]:
LT_GO_DEZ_2019.to_csv('LT_GO_DEZ_2019.csv', index=False)
LT_GO_2020.to_csv('LT_GO_2020.csv', index=False)
LT_GO_2021.to_csv('LT_GO_2021.csv', index=False)
LT_GO_2022.to_csv('LT_GO_2022.csv', index=False)

In [22]:
LT_GO_PAND_COV19 = pd.concat([LT_GO_DEZ_2019, LT_GO_2020, LT_GO_2021, LT_GO_2022], axis=0)

In [23]:
LT_GO_PAND_COV19.to_csv('LT_GO_PAND_COV19.csv', index=False)